#### Reg predict
- Build a linear regression model to look at relationship between different gene attributes and RNA stability

In [ ]:
#Imports
import sys
import os
import pandas as pd
import seaborn as sns
import numpy as np
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

sys.path.append('../scripts')
from plot_helpers import *
from utilities import load_dataset

%load_ext autoreload
%autoreload 2

In [ ]:
#Load stability data
rate_df = load_dataset('../Figures/summary_files/INSPEcT_rates.csv', '../Figures/summary_files/brain4sU_passed.csv')
# CAI_file = '/Users/mkthompson/Desktop/Davislab/3.4_NMJ_4Tu_4sU/3.4e_pipeline_dev/nmj_figures/jupyter_notebooks/Drafting/Figures/gene_attributes/CAI/CAI_values.csv'
# cai_df = pd.read_csv(CAI_file, index_col='gene')
# rate_df['CAI'] = cai_df['CAI_hiexp']

In [ ]:
att_df = pd.read_csv('/Users/mkthompson/Desktop/Davislab/3.4_NMJ_4Tu_4sU/3.4e_pipeline_dev/nmj_figures/jupyter_notebooks/Drafting/Figures/gene_attributes/gene_attributes.csv', index_col=0)

In [ ]:
me_df = pd.read_csv('../Figures/Devreg/gene_cat_me3.csv', index_col='gene')
att_df['category'] = me_df['category']
att_df['TF'] = me_df['TF']
att_df['stab_percentile'] = rate_df['stab_percentile']
#rate_df['updowngene'] = rate_df['category'] == 'updowngene'
#remove non-coding RNAs from the dataset
#this will also remove genes which have no UTR annotated
att_df = att_df.dropna().copy()
att_df['updowngene'] = att_df['category'] == 'updowngene'

In [ ]:
target_column = ['stab_percentile']
# predictors = list(set(list(att_df.columns))-set(target_column))
# predictors = ['category']
# predictors = ['futr_gc', 'tutr_gc']
predictors = ['updowngene', 'TF', 'CAI_hiexp']

In [ ]:
X = att_df[predictors].values
y = att_df[target_column].values

In [ ]:
#Looking at interactions between the variables
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
X_tr = poly.fit_transform(X)
Xt = pd.DataFrame(X_tr, columns=poly.get_feature_names()).drop(['1'],1)
# Xt = pd.concat([X,pd.DataFrame(X_tr,columns=poly.get_feature_names()).drop([‘1’,’x0',’x1',’x2',’x3',’x4'],1)],1)

In [ ]:
X

In [ ]:
X.astype(float)

In [ ]:
y

In [ ]:
# With no interactions
mod = sm.OLS(y, X.astype(float))
res = mod.fit()
res.summary()

In [ ]:
y_pred = res.predict(X.astype(float))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(y, y_pred)

In [ ]:
r2_score(y, y_pred)

In [ ]:
y_pred

In [ ]:
# Now when column 1 is removed
mod = sm.OLS(y, Xt)
res = mod.fit()
res.summary()

In [ ]:
mod = sm.OLS(y, Xt)
res = mod.fit()
res.summary()

In [ ]:
df = pd.DataFrame(X_tr, columns=poly.get_feature_names())

In [ ]:
df.head()

In [ ]:
X_tr

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)
# X2 = ohe.fit_transform(X)

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

In [ ]:
me, tf, cai = lr.coef_.reshape(-1)
intercept = lr.intercept_[0]

In [ ]:
#This doesn't seem to predict accurately though

#Example:
gene_me, gene_tf, gene_cai = [1, 1, 0.7]
gene_me, gene_tf, gene_cai = [0, 0, 0.7]
stab = me*gene_me + tf*gene_tf + cai*gene_cai + intercept

In [ ]:
r2_score(y, lr.predict(X))

In [ ]:
X.shape

In [ ]:
X = np.array(X, dtype=float)

In [ ]:
# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

In [ ]:
#https://groups.google.com/g/pystatsmodels/c/8-4BARFopwc?pli=1

In [ ]:
from statsmodels.formula.api import ols
mod = ols(formula='Stock_Index_Price ~ Interest_Rate + Unemployment_Rate', data=df)
res = mod.fit()
print(res.summary())

In [ ]:
from statsmodels.formula.api import ols
mod = ols(formula='Stock_Index_Price ~ Interest_Rate + Unemployment_Rate + Interest_Rate*Unemployment_Rate', data=df)
res = mod.fit()
print(res.summary())

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary())

In [ ]:
df.head()

In [ ]:
from sklearn import linear_model
import statsmodels.api as sm

#2) Perform linear regression with both sklearn and statsmodels
# here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.
# Alternatively, you may add additional variables within the brackets
Stock_Market = {'Year': [2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016],
                'Month': [12, 11,10,9,8,7,6,5,4,3,2,1,12,11,10,9,8,7,6,5,4,3,2,1],
                'Interest_Rate': [2.75,2.5,2.5,2.5,2.5,2.5,2.5,2.25,2.25,2.25,2,2,2,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75],
                'Unemployment_Rate': [5.3,5.3,5.3,5.3,5.4,5.6,5.5,5.5,5.5,5.6,5.7,5.9,6,5.9,5.8,6.1,6.2,6.1,6.1,6.1,5.9,6.2,6.2,6.1],
                'Stock_Index_Price': [1464,1394,1357,1293,1256,1254,1234,1195,1159,1167,1130,1075,1047,965,943,958,971,949,884,866,876,822,704,719]        
                }

df = pd.DataFrame(Stock_Market,columns=['Year','Month','Interest_Rate','Unemployment_Rate','Stock_Index_Price']) 

X = df[['Interest_Rate','Unemployment_Rate']] 
Y = df['Stock_Index_Price']
 
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

# prediction with sklearn
New_Interest_Rate = 2.75
New_Unemployment_Rate = 5.3
print ('Predicted Stock Index Price: \n', regr.predict([[New_Interest_Rate ,New_Unemployment_Rate]]))


# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)
# sklearn looks more terse, statsmodels more like R

In [ ]:
#intercept here is not meaningful because cai doesn't go below 0.4
lr.intercept_

In [ ]:
?lr

In [ ]:
r2_score(y, lr.predict(X))

In [ ]:
y_pred = lr.predict(X)

In [ ]:
r2_score(y, y_pred)

In [ ]:
lr.coef_

In [ ]:
a = att_df['category'].unique()
a.sort()

In [ ]:
a

In [ ]:
att_df.head()

In [ ]:
att_df['updowngene'] = att_df['category'] == 'updowngene'

In [ ]:
X2[0:5, 2]

In [ ]:
lr.coef_.shape

In [ ]:
len(lr.coef_)

In [ ]:
y_pred.reshape(-1).min()

In [ ]:
y_pred.reshape(-1).max()

In [ ]:
print("Intercept: ", lr.intercept_)
print("Coefficients:")
list(zip(X2, lr.coef_))

In [ ]:
lr.coef_

In [ ]:
X.shape

In [ ]:
X2.shape

In [ ]:
att_df.head()

In [ ]:
print("Intercept: ", mlr.intercept_)
print("Coefficients:")
list(zip(x, mlr.coef_))

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

In [ ]:
y.shape

In [ ]:
y = 
X = rate_df[['CAI', 'updowngene']].values
y = rate_df['stab_percentile'].values

In [ ]:
y.shape

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

In [ ]:
y_pred = lr.predict(X)

In [ ]:
r2_score(y, y_pred)

In [ ]:
pred_train_lr = lr.predict(X_train)
print('RMSE train', np.sqrt(mean_squared_error(y_train, pred_train_lr)))
print('R2 train', r2_score(y_train, pred_train_lr))

In [ ]:
lr.rank_

In [ ]:
?lr